In [17]:
import pandas as pd
pd.options.display.max_columns = None

import numpy as np

In [2]:

base_url = f"https://www.gamedaycalendar.com/schedules/nba/{2023}"

teams = pd.read_html(base_url)
teams_df = pd.concat([division.melt() for division in teams])
teams_df.head()


,variable,value
0,Atlantic,Boston Celtics
1,Atlantic,Brooklyn Nets
2,Atlantic,New York Knicks
3,Atlantic,Philadelphia 76ers
4,Atlantic,Toronto Raptors


In [3]:
seasons = list(range(2014, 2024))
print(seasons)
data_frames = []
for season in seasons:
    # print(season)
    teams = pd.read_html(base_url)
    teams_df = pd.concat([division.melt() for division in teams])
    teams_df['season'] = season
    data_frames.append(teams_df)
# data_frames = [x for x in data_frames if str(type(x)).lower == ""]
division_df = pd.concat(data_frames)
print(division_df['value'].nunique())
print(division_df.shape)
division_df.head(2)

[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
30
(300, 3)


,variable,value,season
0,Atlantic,Boston Celtics,2014
1,Atlantic,Brooklyn Nets,2014


In [4]:
# load team data to get the 3 letter abbreviations for each team
team_info_df = pd.read_csv("../data/team_info.csv")
abbreviations = team_info_df['abbreviation'].str.lower().tolist()
abbreviations[:5]

['atl', 'bos', 'cle', 'nop', 'chi']

In [5]:
# season 2023
url = "https://www.gamedaycalendar.com/schedules/nba/"
months = list(range(12))
dfs = []
errors = []
fixes = {'bkn':"brk", 'phx':"pho", 'sas':"san"}


for season in seasons:
    print(season)
    for team_abbr in abbreviations:
        team_abbr_mod = team_abbr
        if team_abbr_mod in ['bkn', 'phx', 'sas']:
            team_abbr_mod = fixes[team_abbr_mod]
        month_dfs = []
        for month in months:
            try:
                page_data = pd.read_html(
                    f"https://www.gamedaycalendar.com/schedules/nba/{team_abbr_mod}/{season}/{month}"
                )
            except:
                print(f"Error: {season}, {team_abbr}, {month}")
                errors.append(team_abbr)
                continue
            if len(page_data) >= 2:
                temp_df = page_data[1]
                temp_df["team_abbr"] = team_abbr
                temp_df['season'] = season
                month_dfs.append(temp_df)
        if len(month_dfs) > 0:
            dfs.append(pd.concat(month_dfs))

print("Done")
# print(set(errors))
# dfs

2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
Done


In [30]:
game_time_df = pd.concat(dfs)
print(game_time_df.shape)
game_time_df.sample(5)

(25360, 5)


,Date,Time,Opponent,team_abbr,season
11,"Sun, Nov 26",6:00 PM,@ Memphis Grizzlies,bkn,2017
1,"Sat, Nov 2",6:00 PM,vs Charlotte Hornets,gsw,2019
2,"Thu, Nov 3",8:00 PM,vs Boston Celtics,cle,2016
7,"Sat, Dec 13",7:00 PM,vs Atlanta Hawks,orl,2014
2,"Fri, Nov 3",8:30 PM,vs Charlotte Hornets,sas,2017


In [34]:
# do some cleaning

# drop rows that say regular season
game_time_df = game_time_df[
    game_time_df['Date'] != "Regular Season"
].copy()

# split out the date
game_time_df[["day_of_week", "month", "day_of_month"]] = game_time_df['Date'].str.replace(",", "").str.split(" ", expand=True)


# get home vs away
game_time_df["is_home_game"] = np.where(game_time_df['Opponent'].str.contains("@"), 0, 1)

# clean up opponent
# game_time_df['Opponent'] = game_time_df['Opponent'].str.split(" ")

# game_time_df['time_time'] = pd.to_datetime(game_time_df['Time'])



print(game_time_df.shape)
game_time_df.head()

(24100, 9)


,Date,Time,Opponent,team_abbr,season,day_of_week,month,day_of_month,is_home_game
1,"Fri, Jan 2",9:00 PM,@ Utah Jazz,atl,2014,Fri,Jan,2,0
2,"Sat, Jan 3",10:00 PM,@ Portland Trail Blazers,atl,2014,Sat,Jan,3,0
3,"Mon, Jan 5",10:30 PM,@ Los Angeles Clippers,atl,2014,Mon,Jan,5,0
4,"Wed, Jan 7",7:30 PM,vs Memphis Grizzlies,atl,2014,Wed,Jan,7,1
5,"Fri, Jan 9",7:30 PM,@ Detroit Pistons,atl,2014,Fri,Jan,9,0


In [32]:
# game_time_df['Opponent'].unique()

In [7]:
# pd.read_html("https://www.gamedaycalendar.com/schedules/nba/bos/2014/7")#[1]


In [20]:
"vs Altanta Hawks".split(" ", 1)

['vs', 'Altanta Hawks']